In [ ]:
import os
import time
import numpy as np

import pymc3 as pm
import theano
import matplotlib.pyplot as plt

from utils.helpers import *
from utils.plotting import *
from utils.glm_models import *

import sampling.seq_gen as sg
import sampling.seq_analysis as sa

# Import relevant SBL modules
import sbl_agents.sbl_cat_dir as sbl_cd
import sbl_agents.sbl_hmm as sbl_hmm

# Set random seed for replicability and define directories
np.random.seed(seed=1234)
results_dir = os.getcwd() + "/results/"
fig_dir = os.getcwd() + "/figures/"
data_dir = os.getcwd() + "/data/"

In [ ]:
sample, meta = load_obj("data/" + sample_files[0][0] + ".mat")
seq, hidden = sample[:, 2], sample[:, 1]

In [ ]:
# General Model Settings
model_types = ["SP", "AP", "TP"]
save_results = True
verbose = True

sample_files = [["sub-01/sub-01_ses-1_run-1", "sub-01/sub-01_ses-1_run-2",
                 "sub-01/sub-01_ses-1_run-3", "sub-01/sub-01_ses-1_run-4",
                 "sub-01/sub-01_ses-1_run-5"]]

subject_list = range(len(sample_files))

# Load in EEG Dataset
eeg_files = ["sub-01/sub-01_sbl"]
eeg_data = sio.loadmat("data/" + eeg_files[0] + ".mat")

electrodes_of_interest = {"FCz": 47, "FC2": 46, "FC4": 45, 
                          "Cz": 48, "C2": 49, "C4": 50,
                          "C6": 51, "CPz": 32, "CP2": 56,
                          "CP4": 55, "CP6": 54}

trial_coding_lookup = {11: "First Regime - Low Intensity",
                       12: "First Regime - High Intensity",
                       21: "Second Regime - Low Intensity",
                       22: "Second Regime - High Intensity",
                       33: "Catch Trial"}

# Select block and electrode for analysis
subject_id = 0
block_id = 0
elec_id = electrodes_of_interest["FCz"]
sampling_rate = 0.3
inter_stim_interval = np.array([-0.05, 0.65])

- 20 Subjects, 10 Electrodes
- 5 blocks, 10 Different Models, 332 Sample Point per event

In [ ]:
for subject_id in subject_list:
    print("Subject {}: LME Trial-by-Trial Analysis".format(subject_id))
    eeg_data = sio.loadmat("data/" + eeg_files[subject_id] + ".mat")
    
    for block_id, block_fname in enumerate(sample_files):
        print("\t Block {}: Load Seq, Comp Regressors, Load EEG")
        
        # Load in the specific trial/stimuli sequence
        sample, meta = load_obj("data/" + sample_files[subject_id][block_id] + ".mat")
        seq, hidden = sample[:, 2], sample[:, 1]
    
        # Compute Surprise Regressors
        PS_SP, BS_SP, CS_SP = sbl_cd.main(seq, hidden, tau=0, model_type="SP")
        PS_AP, BS_AP, CS_AP = sbl_cd.main(seq, hidden, tau=0, model_type="AP")
        PS_TP, BS_TP, CS_TP = sbl_cd.main(seq, hidden, tau=0, model_type="TP")

        # regressors = {"PS-SP": PS_SP, "BS-SP": BS_SP, "CS-SP": CS_SP,
        #               "PS-AP": PS_AP, "BS-AP": BS_AP, "CS-AP": CS_AP,
        #               "PS-TP": PS_TP, "BS-TP": BS_TP, "CS-TP": CS_TP}

        regressors = {"PS-SP": PS_SP, "BS-SP": BS_SP,
                      "PS-AP": PS_AP, "BS-AP": BS_AP,
                      "PS-TP": PS_TP, "BS-TP": BS_TP}

        # Get the block- and electrode-specific eeg data
        y_elec, y_tw = get_electrode_data(eeg_data, block_id, elec_id,
                                          inter_stim_interval, sampling_rate)
        # Get null model once for a block
        null_model_lme = parallelize_over_samples(y_elec,
                                                  regressor=regressors["PS-AP"],
                                                  reg_model_type="Null")
        for reg_type, regressor in regressors.items():
            print("\t \t Model {}: Run in Parallel".format(reg_model_type))
            start = time.time()
            results_temp = parallelize_over_samples(y_elec, regressor, reg_model_type)
            

In [ ]:
y_labels = ["PS_SP", "BS_SP", "CS_SP",
            "PS_AP", "BS_AP", "CS_AP",
            "PS_TP", "BS_TP", "CS_TP"]
x_labels = ["Block 1", "Block 2", "Block 3", "Block 4"]

heat_data = np.array(list(log_model_evidences.values()))


fig, ax = plt.subplots()
im = ax.imshow(heat_data, cmap="Reds")

# We want to show all ticks...
ax.set_xticks(np.arange(len(x_labels)))
ax.set_yticks(np.arange(len(y_labels)))
# ... and label them with the respective list entries
ax.set_xticklabels(x_labels)
ax.set_yticklabels(y_labels)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

ax.set_title("Log Model Evidences: FCz")

In [ ]:
template = "{} Block | {} GLM Model | Free Energy: {} | Time: {}"

log_model_evidences = {"PS_SP": [], "BS_SP": [], "CS_SP": [],
                       "PS_AP": [], "BS_AP": [], "CS_AP": [],
                       "PS_TP": [], "BS_TP": [], "CS_TP": []}

# Something wrong with events in 5th block!
blocks = np.arange(4)
for block_id in blocks:

    sample, meta = load_obj("data/" + sample_files[0][block_id] + ".mat")
    seq, hidden = sample[:, 2], sample[:, 1]
    
    PS_SP, BS_SP, CS_SP = sbl_cd.main(seq, hidden, tau=0, model_type="SP")
    PS_AP, BS_AP, CS_AP = sbl_cd.main(seq, hidden, tau=0, model_type="AP")
    PS_TP, BS_TP, CS_TP = sbl_cd.main(seq, hidden, tau=0, model_type="TP")

    regressors = {"PS_SP": PS_SP, "BS_SP": BS_SP, "CS_SP": CS_SP,
                  "PS_AP": PS_AP, "BS_AP": BS_AP, "CS_AP": CS_AP,
                  "PS_TP": PS_TP, "BS_TP": BS_TP, "CS_TP": CS_TP}
    
    y_elec = get_electrode_data(eeg_data, block_id, elec_id)
    
    for reg_type, surprise_reg in regressors.items():
        start = time.time()
        free_energy = run_model_estimation(y_elec, surprise_reg, "OLS")
        lme_temp = free_energy[-1]
        log_model_evidences[reg_type].append(lme_temp)
        t_time = time.time() - start
        print(template.format(block_id+1, reg_type, lme_temp, t_time))